# Google Colab (GC) - F2PY CPU sequencial

In [1]:
%load_ext fortranmagic

/usr/local/lib/python3.7/dist-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


In [10]:
%%fortran --f90flags "-O3"
subroutine st(n, energy, niters, heat, t)
    integer, intent(in) :: n, energy, niters
    double precision, intent(out) :: heat, t   
    integer, parameter :: nsources=3
    integer :: iters, i, j, x, y, sizeArray, sizeStart, sizeEnd
    integer, dimension(3, 2) :: sources
    double precision, allocatable :: aold(:,:), anew(:,:)
    double precision :: t1=0.0, t2=0.0

    call cpu_time(t1)

    sizeArray = n + 2
    sizeStart = 2
    sizeEnd = n + 1

    allocate(aold(sizeArray, sizeArray))
    allocate(anew(sizeArray, sizeArray))
    aold = 0.0
    anew = 0.0
    
    sources(1,:) = (/ n/2,   n/2   /)
    sources(2,:) = (/ n/3,   n/3   /)
    sources(3,:) = (/ n*4/5, n*8/9 /)
    
    do iters = 1, niters, 2
        do j = sizeStart, sizeEnd
            do i = sizeStart, sizeEnd
                anew(i,j) = aold(i,j)/2.0 + (aold(i-1,j) + aold(i+1,j) +  &
                            aold(i,j-1) + aold(i,j+1)) / 4.0 / 2.0
            enddo
        enddo
        do i = 1, nsources
            x = sources(i,1) + 1
            y = sources(i,2) + 1
            anew(x,y) =  anew(x,y) + energy
        enddo
        do j = sizeStart, sizeEnd
            do i = sizeStart, sizeEnd
                aold(i,j) = anew(i,j)/2.0 + (anew(i-1,j) + anew(i+1,j) +  &
                            anew(i,j-1) + anew(i,j+1)) / 4.0 / 2.0
            enddo
        enddo
        do i = 1, nsources
            x = sources(i,1) + 1
            y = sources(i,2) + 1
            aold(x,y) = aold(x,y) + energy
        enddo
    enddo
    heat = 0.0
    do j = sizeStart, sizeEnd
        do i = sizeStart, sizeEnd
            heat = heat + aold(i,j)
        end do
    end do
    deallocate(aold)
    deallocate(anew)
    call cpu_time(t2)
    t = t2 - t1
end subroutine

In [8]:
from f2py_seq import st
print(st.__doc__)

heat,t = st(n,energy,niters)

Wrapper for ``st``.

Parameters
----------
n : input int
energy : input int
niters : input int

Returns
-------
heat : float
t : float



In [9]:
%%timeit -n1 -r1
from time import time
from f2py_seq import st

n       = 4800    # nxn grid; 4800,1,500→1500; 100,1,10→30; [4800]
energy  = 1       # energy to be injected per iteration; [1]
niters  = 500     # number of iterations; [500]
t0      = -time()

heat, t = st(n, energy, niters)

t0 += time()
print("Heat = %0.4f | Time = %0.4f | TimePyt = %0.4f" %(heat, t, t0))

Heat = 1500.0000 | Time = 17.0064 | TimePyt = 17.0402
17 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
